In [1]:
from neuronunit.optimization import get_neab
# http://www.physics.usyd.edu.au/teach_res/mp/mscripts/
# ns_izh002.m
import collections
from collections import OrderedDict

# Fast spiking cannot be reproduced as it requires modifications to the standard Izhi equation,
# which are expressed in this mod file.
# https://github.com/OpenSourceBrain/IzhikevichModel/blob/master/NEURON/izhi2007b.mod

reduced2007 = collections.OrderedDict([
  #              C    k     vr  vt vpeak   a      b   c    d  celltype
  ('RS',        (100, 0.7,  -60, -40, 35, 0.03,   -2, -50,  100,  1)),
  ('IB',        (150, 1.2,  -75, -45, 50, 0.01,   5, -56,  130,   2)),
  ('LTS',       (100, 1.0,  -56, -42, 40, 0.03,   8, -53,   20,   4)),
  ('TC',        (200, 1.6,  -60, -50, 35, 0.01,  15, -60,   10,   6)),
  ('TC_burst',  (200, 1.6,  -60, -50, 35, 0.01,  15, -60,   10,   6)),
  ('RTN',       (40,  0.25, -65, -45,  0, 0.015, 10, -55,   50,   7)),
  ('RTN_burst', (40,  0.25, -65, -45,  0, 0.015, 10, -55,   50,   7))])

import numpy as np
reduced_dict = OrderedDict([(k,[]) for k in ['C','k','vr','vt','vPeak','a','b','c','d']])

#OrderedDict
for i,k in enumerate(reduced_dict.keys()):
    for v in reduced2007.values():
        reduced_dict[k].append(v[i])
        
reduced_cells = OrderedDict([(k,[]) for k in ['RS','IB','LTS','TC','TC_burst']])

for index,key in enumerate(reduced_cells.keys()):
    reduced_cells[key] = {}
    for k,v in reduced_dict.items():
        reduced_cells[key][k] = v[index]        

explore_param = {k:(np.min(v),np.max(v)) for k,v in reduced_dict.items()}

In [2]:
purkinje ={"id": 18, "name": "Cerebellum Purkinje cell", "neuron_db_id": 271, "nlex_id": "sao471801888"}
fi_basket = {"id": 65, "name": "Dentate gyrus basket cell", "neuron_db_id": None, "nlex_id": "nlx_cell_100201"}
pvis_cortex = {"id": 111, "name": "Neocortex pyramidal cell layer 5-6", "neuron_db_id": 265, "nlex_id": "nifext_50"}
#does not have rheobase
#olf_mitral = {"id": 129, "name": "Olfactory bulb (main) mitral cell", "neuron_db_id": 267, "nlex_id": "nlx_anat_100201"}
ca1_pyr = {"id": 85, "name": "Hippocampus CA1 pyramidal cell", "neuron_db_id": 258, "nlex_id": "sao830368389"}
pipe = [ fi_basket, ca1_pyr, purkinje,  pvis_cortex]

In [3]:
electro_tests = []
electro_frame = {}

#p_tests, p_observations = get_neab.get_neuron_criteria(olf_mitral)
#electro_frame[p["name"]] = p_observations#, p_tests))


In [4]:
electro_tests = []
obs_frame = {}
test_frame = {}
import os
import pickle
try: 

    electro_path = str(os.getcwd())+'all_tests.p'
    assert os.path.isfile(electro_path) == True
    with open(electro_path,'rb') as f:
        (obs_frame,test_frame) = pickle.load(f)

except:
    for p in pipe:
        p_tests, p_observations = get_neab.get_neuron_criteria(p)
        obs_frame[p["name"]] = p_observations#, p_tests))
        test_frame[p["name"]] = p_tests#, p_tests))
    electro_path = str(os.getcwd())+'all_tests.p'
    with open(electro_path,'wb') as f:
        pickle.dump((obs_frame,test_frame),f)

print(test_frame)
    

{'Hippocampus CA1 pyramidal cell': [<neuronunit.tests.fi.RheobaseTest object at 0x7f7d40fedb38>, <neuronunit.tests.passive.InputResistanceTest object at 0x7f7d40fedb70>, <neuronunit.tests.passive.TimeConstantTest object at 0x7f7d40fedba8>, <neuronunit.tests.passive.CapacitanceTest object at 0x7f7d40fedbe0>, <neuronunit.tests.passive.RestingPotentialTest object at 0x7f7d40fedc18>, <neuronunit.tests.waveform.InjectedCurrentAPWidthTest object at 0x7f7d40fedc50>, <neuronunit.tests.waveform.InjectedCurrentAPAmplitudeTest object at 0x7f7d40fedc88>, <neuronunit.tests.waveform.InjectedCurrentAPThresholdTest object at 0x7f7d40fedcc0>], 'Cerebellum Purkinje cell': [<neuronunit.tests.fi.RheobaseTest object at 0x7f7d40fed9b0>, <neuronunit.tests.passive.InputResistanceTest object at 0x7f7d40fed9e8>, <neuronunit.tests.passive.CapacitanceTest object at 0x7f7d40feda20>, <neuronunit.tests.passive.RestingPotentialTest object at 0x7f7d40feda58>, <neuronunit.tests.waveform.InjectedCurrentAPWidthTest objec

In [5]:
import pandas as pd
from neuronunit.tests.fi import RheobaseTestP
#obs_frame.pop("olf_mit", 0)
#test_frame.pop("olf_mit", 0)

for k,v in test_frame.items():
    if "olf_mit" not in k:
        obs = obs_frame[k]
        v[0] = RheobaseTestP(obs['Rheobase'])


df = pd.DataFrame.from_dict(obs_frame)

print(test_frame.keys())
df['Hippocampus CA1 pyramidal cell']

dict_keys(['Hippocampus CA1 pyramidal cell', 'Cerebellum Purkinje cell', 'Neocortex pyramidal cell layer 5-6', 'olf_mit', 'Dentate gyrus basket cell'])


Cell Capacitance              {'std': 53.6497639516366 pF, 'mean': 89.796071...
Input Resistance              {'std': 77.577415348574 Mohm, 'mean': 107.0803...
Membrane Time Constant        {'std': 21.4877858716633 ms, 'mean': 24.502194...
Resting membrane potential    {'std': 4.88864451466597 mV, 'mean': -65.22618...
Rheobase                      {'std': 287.163664156461 pA, 'mean': 189.24 pA...
Spike Amplitude               {'std': 12.2626088055275 mV, 'mean': 86.364525...
Spike Half-Width              {'std': 0.608307582372384 ms, 'mean': 1.318952...
Spike Threshold               {'std': 7.7267055100184 mV, 'mean': -47.598571...
Name: Hippocampus CA1 pyramidal cell, dtype: object

In [6]:
##### 
from neuronunit.optimization import optimization_management as om
import pickle


free_params = ['a','b','k','c','C','d','vPeak','vr']#vt','c','k','d']#,'vt','k','c','C']#,'C'] # this can only be odd numbers.

##
# Use information that is available
##
hc_ = reduced_cells['RS']

hc_['vr'] = -65.2261863636364

hc_['vPeak'] = hc_['vr'] + 86.364525297619

explore_param['C'] = (hc_['C']-20,hc_['C']+20)
explore_param['vr'] = (hc_['vr']-5,hc_['vr']+5)
#hc = {}
#hc['C'] = 89.7960714285714


use_test = test_frame["Neocortex pyramidal cell layer 5-6"]

from sciunit import scores# score_type 

#for t in use_test[::-1]:
#    t.score_type = scores.RatioScore
#print(use_test)
reduced_tests = [use_test[0], use_test[-2], use_test[len(use_test)-1]]
bigger_tests = use_test[1:-2]
bigger_tests.insert(0,use_test[0])

test_opt = {}
import pickle

with open('data_dump.p','wb') as f:
    pickle.dump(test_opt,f)


    


In [ ]:

MU = 12
NGEN = 20
#try:
#    with open('multi_objective.p','rb') as f:
#        test_opt = pickle.load(f)    
#except:
npcl, DO = om.run_ga(explore_param,NGEN,use_test,free_params=free_params, NSGA = True, MU = MU)
print('can get as low as 2.84684')

test_opt[str(t)] =  {'multi_objective':npcl}
with open('multi_objective.p','wb') as f:
    pickle.dump(test_opt,f)

/opt/conda/lib/python3.5/site-packages/quantities/quantity.py:414: RuntimeWarning: invalid value encountered in greater
  return self.magnitude > other
/opt/conda/lib/python3.5/site-packages/quantities/quantity.py:414: RuntimeWarning: invalid value encountered in greater
  return self.magnitude > other
/opt/conda/lib/python3.5/site-packages/quantities/quantity.py:414: RuntimeWarning: invalid value encountered in greater
  return self.magnitude > other
/opt/conda/lib/python3.5/site-packages/quantities/quantity.py:414: RuntimeWarning: invalid value encountered in greater
  return self.magnitude > other
/opt/conda/lib/python3.5/site-packages/quantities/quantity.py:414: RuntimeWarning: invalid value encountered in greater
  return self.magnitude > other
/opt/conda/lib/python3.5/site-packages/quantities/quantity.py:414: RuntimeWarning: invalid value encountered in greater
  return self.magnitude > other
/opt/conda/lib/python3.5/site-packages/quantities/quantity.py:414: RuntimeWarning: inval

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


/home/jovyan/neuronunit/neuronunit/tests/passive.py:82: RuntimeWarning: overflow encountered in exp
  vm_fit[offset:,0] = a * np.exp(-t[offset:]/b) + c
/opt/conda/lib/python3.5/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


1
1


/opt/conda/lib/python3.5/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


...

[[120, 0.010833505646215302, 13.660294871104041, -60.0, 37.444246128114173, 0.25, 30.27763762604652, -60.226186363636401], [120, 0.01, -2.0, -60.0, 130, 0.25, 50.0, -60.226186363636401], [109.59183752636488, 0.01, -2, -60, 130.0, 0.25342759284448141, 50, -60.226186363636401], [80.0, 0.01, -2, -50, 10, 0.26973290859758769, 50.0, -60.226186363636401], [80.0, 0.01, -2, -50, 10.0, 0.26973290859758769, 50.0, -60.226186363636401], [119.78265116076513, 0.010999180959208541, 1.8975051526185309, -60.0, 105.61231025453614, 0.25, 38.233943044172889, -60.263630379311415], [80, 0.01, 15, -60.0, 130, 1.6000000000000001, 48.232509244231828, -70.226186363636401], [120, 0.01008336047927072, 14.611853327863724, -60, 37.444246128114173, 0.25, 50, -69.809030946071957], [90.114853748563931, 0.01, -2.0, -50, 10, 0.26973290859758769, 49.118096735597241, -60.226186363636401], [118.48783386580797, 0.01297469478811761, 14.221722302432383, -50.359268694777853, 13.578488009363028, 0.39637236117369473, 4.177620043

/opt/conda/lib/python3.5/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


1
1
1
1
1
1
1
1
1
1
1
1


INFO:__main__:6  	12    	5.54561	0.574546	4.51868	6.47622


[[120, 0.01, -2.0, -60.0, 130, 0.25, 50.0, -60.226186363636401], [109.59183752636488, 0.01, -2, -60, 130.0, 0.25342759284448141, 50, -60.226186363636401], [80.0, 0.01, -2, -50, 10, 0.26973290859758769, 50.0, -60.226186363636401], [80.0, 0.01, -2, -50, 10.0, 0.26973290859758769, 50.0, -60.226186363636401], [80, 0.01, 15, -60.0, 130, 1.6000000000000001, 48.232509244231828, -70.226186363636401], [120, 0.010787991483458374, -1.5719728229360808, -60.0, 32.645402842132881, 0.25, 37.446456784067877, -60.226186363636401], [120, 0.010068185369671714, 14.587248080700368, -58.96807557309419, 128.60660324498673, 0.27741248267177782, 49.664871986686769, -60.226186363636401], [47.101178486702402, 0.025604694240664704, -2, -60.0, 160.59715420978273, 0.61416695228875895, 9.3301026485098024, -66.352213358762455], [47.101178486702402, 0.024448333799169795, -11.149380662328371, -62.794484183503563, 160.59715420978273, 0.72435290120416895, 4.199917364455203, -66.766607111794599], [47.101178486702402, 0.02

In [ ]:
MU = 10
NGEN = 10
cnt = 1
for t in use_test:        
    if cnt==len(use_test):
        MU = 12
        NGEN = 20

        npcl, DO = om.run_ga(explore_param,NGEN,[t],free_params=free_params, NSGA = True, MU = MU)
    else:

        npcl, DO = om.run_ga(explore_param,NGEN,[t],free_params=free_params, NSGA = True, MU = MU)

    test_opt[str(t)] =  {'out':npcl}
    
    ranges = {}
    stds = npcl['pop'][0].dtc.attrs
    for k in npcl['pop'][0].dtc.attrs.keys():    
        stds[k] = []
        ranges[k] = []
        
        
    for i in npcl['pop'][::5]:
        for k,v in i.dtc.attrs.items():
            stds[k].append(v)
            ranges[k].append(v)

    for k in npcl['pop'][0].dtc.attrs.keys():
        ranges[k] = (np.min(ranges[k][1::]),np.max(ranges[k][1::]))

        stds[k] = np.std(stds[k][1::])
    test_opt[str(t)]['stds'] = stds 
    test_opt[str(t)]['ranges'] = ranges 

    with open('data_dump.p','wb') as f:
        pickle.dump(test_opt,f)
    cnt+=1

I consider the final gene populations for each of the eight tests. I compute the variance in each of the converged populations, I see that variance is low in many of the gene populations.

When all variables are used to optomize only against one set of parameters, you expect their would be high variance in parameters, that don't matter much with respect to that error criteria (you expect redundancy of solutions).

I compute std on errors over all the tests in order to estimate how amenable the problem is to multiobjective optimization.

In [ ]:
print(test_opt['t'][0]['pf'])

In [ ]:
#print([i.fitness.values for i in test_opt['t'][0]['pop']])#.keys()
print(np.std([i[0] for i in test_opt['t'][0]['pop'][0:5]]))#.keys()
print(np.std([i[1] for i in test_opt['t'][0]['pop'][0:5]]))#.keys()
print(np.std([i[2] for i in test_opt['t'][0]['pop'][0:5]]))#.keys()
print(np.std([i[3] for i in test_opt['t'][0]['pop'][0:5]]))#.keys()
print(test_opt['t'][0]['pop'][0][0])
print(test_opt['t'][0]['pop'][0][1])
test_opt['t'][0]['pop'][0].dtc.attrs

In [ ]:

#values = { k:v for v in npcl['pop'][i].dtc.attrs.items() for i in npcl['pop'] }
print(values)    
print(stds.keys())
stds
#dtc.variances[k] for k in dtc.attrs.keys() 

In [ ]:

DO.seed_pop = npcl['pf'][0:MU]
npcl, DO = om.run_ga(explore_param,10,reduced_tests,free_params=free_params,hc = hc, NSGA = False, MU = MU, seed_pop = DO.seed_pop)


In [ ]:
attrs_here = npcl['hardened'][0][0].attrs
attrs_here.update(hc)
attrs_here
scores = npcl['hof'][0].dtc.scores
print(scores)

In [ ]:
#
use_test = test_frame["Neocortex pyramidal cell layer 5-6"]
reduced_tests = [use_test[0], use_test[-1], use_test[len(use_test)-1]]
bigger_tests = use_test[1:-2]
bigger_tests.insert(0,use_test[0])

In [ ]:
#bigger_tests = bigger_tests[-1::]
print(bigger_tests)

In [ ]:
DO.seed_pop = npcl['hof'][0:MU]
reduced_tests = [use_test[0], use_test[-1], use_test[len(use_test)-1]]
npcl, DO = om.run_ga(explore_param,10,bigger_tests,free_params=free_params,hc = hc, NSGA = False, MU = MU)#, seed_pop = DO.seed_pop)


In [ ]:
print(npcl['hardened'][0][0].attrs)
print(npcl['hardened'][0][0].scores)
print(npcl['pf'][0].fitness.values)
print(npcl['hof'][0].dtc.scores)

#for t in use_test:
#    print(t.name)
    
    
pop

# From the scores printed above, it looks like certain error criteria, are in conflict with each other.

Tests, that are amenable to co-optimization appear to be:
* Width test
* Input resistance tets
* Resting Potential Test,
* Capicitance test.
* Time constant

Tests/criteria that seem in compatible with the above include: 
* Rheobase, 
* InjectedCurrentAPThresholdTest
* InjectedCurrentAPAmplitudeTest

Therefore a reduced set of lists is made to check if the bottom three are at least amenable to optimization togethor.

In [ ]:
from sklearn.cluster import KMeans
est = KMeans(n_clusters=2)
est.fit(X)
y_kmeans = est.predict(X)

centers = est.cluster_centers_

fig = plt.figure(fignum,figsize=(4,3))
ax = Axes3D(fig,rect=[0,0,.95,1],elav=48,azim=134)
ax.scatter(X[:,0],X[:,1],X[:,2],c=y_kmeans,s=50),
ax.scatter(centres[:,0],centres[:,1],centres[:,2],c='black',s=200,alpha=0.5)

In [ ]:

print(reduced_tests)
print(bigger_tests)

DO.seed_pop = npcl['pf'][0:MU]
npcl, DO = om.run_ga(explore_param,10,reduced_tests,free_params=free_params,hc = hc, NSGA = True, MU = 12)#, seed_pop = DO.seed_pop)


In [ ]:
import pickle
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from neuronunit.optimization.optimization_management import wave_measure
from neuronunit.models.reduced import ReducedModel
from neuronunit.optimization.model_parameters import model_params, path_params
LEMS_MODEL_PATH = path_params['model_path']
import neuronunit.optimization as opt
import quantities as pq
fig = plt.figure()

plt.clf()

from neuronunit.optimization.data_transport_container import DataTC
model = ReducedModel(LEMS_MODEL_PATH,name = str('vanilla'),backend = ('RAW'))
for i in npcl['pf'][0:2]:
    iparams = {}
    iparams['injected_square_current'] = {}
    iparams['injected_square_current']['amplitude'] =i.dtc.rheobase
    model = None
    model = ReducedModel(LEMS_MODEL_PATH,name = str('vanilla'),backend = ('RAW'))
    model.set_attrs(i.dtc.attrs)

    #['amplitude']  = dtc.vtest[k]['injected_square_current']['amplitude']
    DELAY = 100.0*pq.ms
    DURATION = 1000.0*pq.ms
    iparams['injected_square_current']['delay'] = DELAY
    iparams['injected_square_current']['duration'] = int(DURATION)
    model.inject_square_current(iparams)
    n_spikes = len(model.get_spike_train())
    if n_spikes:
        print(n_spikes)
        #print(i[0].scores['RheobaseTestP']*pq.pA)
        plt.plot(model.get_membrane_potential().times,model.get_membrane_potential())#,label='ground truth')
        plt.legend()

#gca().set_axis_off()
#subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
#            hspace = 0, wspace = 0)
#margins(0,0)
#gca().xaxis.set_major_locator(NullLocator())
#gca().yaxis.set_major_locator(NullLocator())

plt.subplots_adjust(left=0.0, right=1.0, top=0.9, bottom=0.1)
fig.tight_layout()
plt.show()

fig.savefig("single_trace.png", bbox_inches = 'tight',
    pad_inches = 0)


In [ ]:
import pickle
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot a...s plt

from neuronunit.models.reduced import ReducedModel
from neuronunit.optimization.model_parameters import model_params, path_params
LEMS_MODEL_PATH = path_params['model_path']
import neuronunit.optimization as opt
import quantities as pq
fig = plt.figure()

plt.clf()

from neuronunit.optimization.data_transport_container import DataTC
for i in npcl['hardened']:
    iparams = {}
    iparams['injected_square_current'] = {}
    iparams['injected_square_current']['amplitude'] = i[0].rheobase
    model = None
    model = ReducedModel(LEMS_MODEL_PATH,name = str('vanilla'),backend = ('RAW'))
    model.set_attrs(i[0].attrs)

    #['amplitude']  = dtc.vtest[k]['injected_square_current']['amplitude']
    DELAY = 100.0*pq.ms
    DURATION = 1000.0*pq.ms
    iparams['injected_square_current']['delay'] = DELAY
    iparams['injected_square_current']['duration'] = int(DURATION)
    model.inject_square_current(iparams)
    n_spikes = len(model.get_spike_train())
    if n_spikes:
        print(n_spikes)
        print(i[0].scores['RheobaseTestP']*pq.pA)
        plt.plot(model.get_membrane_potential().times,model.get_membrane_potential())#,label='ground truth')
        plt.legend()

#gca().set_axis_off()
#subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
#            hspace = 0, wspace = 0)
#margins(0,0)
#gca().xaxis.set_major_locator(NullLocator())
#gca().yaxis.set_major_locator(NullLocator())

plt.subplots_adjust(left=0.0, right=1.0, top=0.9, bottom=0.1)
fig.tight_layout()
plt.show()

fig.savefig("single_trace.png", bbox_inches = 'tight',
    pad_inches = 0)


'''
hc = {}

#free_params = ['c','k']
for k,v in explore_param.items():
    if k not in free_params:
        hc[k] = v
constants = npcl['hardened'][0][0].attrs
hc.update(constants) 
npcl, _ = om.run_ga(explore_param,20,test_frame["Neocortex pyramidal cell layer 5-6"],free_params=free_params,hc = hc, NSGA = True)
'''

In [ ]:

free_params = ['a','b','k']#vt','c','k','d']#,'vt','k','c','C']#,'C'] # this can only be odd numbers.

##
# Use information that is available
##
hc = reduced_cells['RS']

hc['vr'] = -65.2261863636364

hc['vPeak'] = hc['vr'] + 86.364525297619
hc['C'] = 89.7960714285714
hc.pop('a',0)
hc.pop('b',0)
hc.pop('k',0)
hc.pop('c',0)
hc.pop('d',0)
        
use_test = test_frame["Neocortex pyramidal cell layer 5-6"]
DO.seed_pop = npcl['pf']
ga_out = DO.run(max_ngen = 15)
'''
hc = {}

free_params = ['C']

for k,v in explore_param.items():
    if k not in free_params:
        hc[k] = v
#,'vt','k','c','C']#,'C'] # this can only be odd numbers
constants = npcl['hardened'][0][0].attrs
hc.update(constants) 
npcl, _ = om.run_ga(explore_param,20,test_frame["Neocortex pyramidal cell layer 5-6"],free_params=free_params,hc = hc, NSGA = True)
'''

In [ ]:
'''
import pandas
    
try:
    ne_raw = pandas.read_csv('article_ephys_metadata_curated.csv', delimiter='\t')
    !ls -ltr *.csv
except:
    !wget https://neuroelectro.org/static/src/article_ephys_metadata_curated.csv
    ne_raw = pandas.read_csv('article_ephys_metadata_curated.csv', delimiter='\t')

blah = ne_raw[ne_raw['NeuronName'].str.match('Hippocampus CA1 pyramidal cell')]
#ne_raw['NeuronName']
#ne_raw['cell\ capacitance']
#blah = ne_raw[ne_raw['NeuronName'].str.match('Hippocampus CA1 pyramidal cell')]

print([i for i in blah.columns])
#rint(blah['rheobase'])
#print(blah)
#for i in ne_raw.columns:#['NeuronName']:
#    print(i)

#ne_raw['NeuronName'][85]
#blah = ne_raw[ne_raw['TableID'].str.match('85')]
#ne_raw['n'] = 84
#here = ne_raw[ne_raw['Index']==85]
here = ne_raw[ne_raw['TableID']==18]

print(here['rheo_raw'])
#!wget https://neuroelectro.org/apica/1/n/
'''

In [ ]:
ca1 = ne_raw[ne_raw['NeuronName'].str.match('Hippocampus CA1 pyramidal cell')]
ca1['rheo']

In [ ]:

    
test_frame["Dentate gyrus basket cell"][0].observation['std'] = test_frame["Dentate gyrus basket cell"][0].observation['mean']
for t in test_frame["Dentate gyrus basket cell"]:
    print(t.name)

    print(t.observation)
    


'''
Inibitory Neuron
This can't pass the Rheobase test
'''    

In [ ]:

from neuronunit.optimization import optimization_management as om
import pickle

free_params = ['a','vr','b','vt','vPeak','c','k']
for k,v in explore_param.items():
    if k not in free_params:
        hc[k] = v
use_test = test_frame["Dentate gyrus basket cell"]
bcell, _ = om.run_ga(explore_param,20,use_test,free_params=free_params,hc = hc, NSGA = True, MU = 4)


In [ ]:

    
#test_frame["Dentate gyrus basket cell"][0].observation['std'] = test_frame["Dentate gyrus basket cell"][0].observation['mean']
for t in test_frame["Hippocampus CA1 pyramidal cell"]:
    print(t.name)

    print(t.observation)

In [ ]:
use_test = test_frame["Hippocampus CA1 pyramidal cell"]
bcell, _ = om.run_ga(explore_param,20,use_test,free_params=free_params,hc = hc, NSGA = True, MU = 10)


In [ ]:
import pickle
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from neuronunit.models.reduced import ReducedModel
from neuronunit.optimization.model_parameters import model_params, path_params
LEMS_MODEL_PATH = path_params['model_path']
import neuronunit.optimization as opt
import quantities as pq
fig = plt.figure()

plt.clf()

from neuronunit.optimization.data_transport_container import DataTC
for i in bcell['hardened'][0:6]:
    iparams = {}
    iparams['injected_square_current'] = {}
    iparams['injected_square_current']['amplitude'] =i[0].rheobase
    model = None
    model = ReducedModel(LEMS_MODEL_PATH,name = str('vanilla'),backend = ('RAW'))
    model.set_attrs(i[0].attrs)

    #['amplitude']  = dtc.vtest[k]['injected_square_current']['amplitude']
    DELAY = 100.0*pq.ms
    DURATION = 1000.0*pq.ms
    iparams['injected_square_current']['delay'] = DELAY
    iparams['injected_square_current']['duration'] = int(DURATION)
    model.inject_square_current(iparams)
    n_spikes = len(model.get_spike_train())
    if n_spikes:
        print(n_spikes)
        print(i[0].scores['RheobaseTestP']*pq.pA)
        plt.plot(model.get_membrane_potential().times,model.get_membrane_potential())#,label='ground truth')
        plt.legend()

#gca().set_axis_off()
#subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
#            hspace = 0, wspace = 0)
#margins(0,0)
#gca().xaxis.set_major_locator(NullLocator())
#gca().yaxis.set_major_locator(NullLocator())

plt.subplots_adjust(left=0.0, right=1.0, top=0.9, bottom=0.1)
fig.tight_layout()
plt.show()

fig.savefig("single_trace.png", bbox_inches = 'tight',
    pad_inches = 0)


In [ ]:
use_test = test_frame["Hippocampus CA1 pyramidal cell"]
bcell, _ = om.run_ga(explore_param,20,use_test,free_params=free_params,hc = hc, NSGA = True, MU = 10)
